In [8]:
import pandas as pd
import json
from tqdm import tqdm

# импортируем библиотеку для работы с картами
# from keplergl import KeplerGl

from db_parameters import *

In [2]:
from ParseLog import ParseLog

In [3]:
p_py = ParseLog('log/20220511T174759.log')

In [4]:
df = p_py.transform_df()
df

,timestamp,clid,uuid,route,vehicle_type,latitude,longitude,speed,direction,production,garange_number,reg_number,route_descr
0,2022-05-11 16:01:20,cherepovets,ak1456_2058,#7,bus,59.13450241088867,37.923988342285156,15.17,192,1,,е440хк_35 [БВ (Н)],7 [ул. олимпийская - > - ул. наседкина]\n
1,2022-05-11 16:01:19,cherepovets,ak1456_1881,#,bus,59.1339225769043,37.97321701049805,0.00,0,0,,е254хк_35 [БВ (Н)],\n
2,2022-05-11 16:01:17,cherepovets,ak1456_2424,#7,bus,59.12198257446289,37.97989273071289,9.44,8,1,,е401хк_35 [БВ (Н)],7 [ул. олимпийская - > - ул. наседкина]\n
3,2022-05-11 16:01:17,cherepovets,ak1456_1946,#27,bus,59.096702575683594,37.9358024597168,28.49,280,1,,е387хк_35 [БВ (Н)],27 [ул. монт - клер - > - доменная]\n
4,2022-05-11 16:01:18,cherepovets,ak1456_1892,#8,bus,59.126468658447266,37.91294479370117,17.76,103,1,,е329хк_35 [БВ (Н)],8 [доменная - > - ул. красная]\n
...,...,...,...,...,...,...,...,...,...,...,...,...,...
67596,2022-05-11 17:47:56,cherepovets,novotrans_1146909,#37,bus,59.124255,37.9060116667,9.00,103,1,,,37 зима*_февраль Доменная — Ветеранов\n
67597,2022-05-11 17:47:50,cherepovets,novotrans_200040110,#25,bus,59.1014066667,37.908485,14.00,145,1,,,25*_ФЕВРАЛЬ Доменная — Рыбинская\n
67598,2022-05-11 17:47:54,cherepovets,novotrans_14493723,#38,bus,59.1268466,37.9565166,31.00,282,1,,,38_февраль ГОГОЛЯ - ДОМЕННАЯ\n
67599,2022-05-11 17:47:54,cherepovets,novotrans_14867608,#,bus,59.136538,37.977195,67.00,164,0,,,\n


In [42]:
df.longitude = df.longitude.astype(float)
df.latitude = df.latitude.astype(float)
df.speed = df.speed.astype(float)
df.direction = df.direction.astype(int)
df.production = df.production.astype(int)

In [40]:
float(df.latitude[0])

59.13450241088867

In [25]:
from sqlalchemy import create_engine

In [31]:
def authorization_pg():
    """
    Создание коннектора к БД PostgreSQL
    """
    login_name = LOGIN
    login_password = PASSWORD
    IP = HOST
    engine_str = f'postgresql://{login_name}:{login_password}@{IP}:5432/{DB}'
    engine = create_engine(engine_str)
    return engine

In [32]:
def generate(df):
    """
    Создания словаря для заполнения таблицы в БД PostgreSQL
    """
    d = {
        'timestamp':df.timestamp,
        'clid':df.clid,
        'uuid':df.uuid,
        'route':df.route,
        'vehicle_type':df.vehicle_type,
        'latitude':df.latitude,
        'longitude':df.longitude,
        'speed':df.speed,
        'direction':df.direction,
        'production':df.production,
        'garange_number':df.garange_number,
        'reg_number':df.reg_number,
        'route_descr':df.route_descr
    }
    df_into = pd.DataFrame(data=d)
    return df_into

In [33]:
def into_pg_table(pg_table_name,df):
    """
    Помещение данных в БД PostgreSQL
    """
    dataframe = generate(df)
    connector = authorization_pg()
    dataframe.to_sql(
        name = pg_table_name,
        con = connector,
        chunksize = 10000,
        index = False,
        if_exists = 'append'
    )

In [35]:
%%time

vehicle_type = ['bus', 'tramway', 'minibus', 'trolleybus']

for veh in vehicle_type:
    df_into = df[df.vehicle_type==veh]
    into_pg_table(veh,df_into)

CPU times: user 3.31 s, sys: 127 ms, total: 3.43 s
Wall time: 11 s


Наиcolumnsие колонок:

- `TIMESTAMP` - время в UTC в формате дата:время
- `CLID` - vologda или cherepovets
- `UUID` - уникальный идентификатор ТС в формате id_перевозчика + "_" + id_ТС_в базе_перевозчика
- `ROUTE` - "#" + номер маршрута 
- `VEHICLE_TYPE` - 'bus', 'tramway', 'minibus', 'trolleybus'
- `LATITUDE` – широта
- `LONGITUDE` – долгота
- `SPEED` - текущая скорость
- `DIRECTION` - азимут
- `PRODUCTION` - признак производственного рейса. Когда "0" - скорее всего автобус без пассажиров. В Яндекс передаются только данные с флагом "1"
- `GARANGE_NUMBER` - опционально "человеческий" идентификатор ТС для перевозчика
- `REG_NUMBER` - госномер ТС, есть не всегда
- `ROUTE_DESCR` - текстовый идентификатор траектории маршрута в одном направлении. Нет стандартного формата, можно использовать только как "хэш", как ключ для группировки точек по траекториям

# Создание карты

In [94]:
with open('config.json') as f:
    config = json.load(f)

_map = KeplerGl()
_map.config = config

_map.add_data(data=df_draw, name='yelyi5ow8')

_map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [{'dataId': ['yelyi5ow8'], 'id': '3ynfdw2…